# Train a recurrent neural network to predict the position of the mouse in the v0,v1 coordinate system.

We train a model for each recording session.

We get the animal position in the v0,v1 coordinate system from `transform_xy_to_grid_representation()`.

We get the instantaneous firing rate of all our grid cells. 

We need to remove time-points with invalid values as the model can't deal with np.nan.


### Rate normalization

The firing rate of neuron is normalize for two reasons: all neurons have a similar contribution and values around 0 should facilitate learning. For each neuron, the rate vector is transform so that its mean is 0 and STD is 1. 

When doing inference, we will use the same transformation as during training to normalize the firing rate. This is because we often have a very poor estimate of the firing rate from short streches of data used for inference.

### v0,v1 as model labels

Each component of v0,v1 coordinate is periodic (circular data). To facilite learning, We decompose $v_{i}$ into $cos(v_{i})$ and $sin(v_{i})$. This way the values will range from -1 to 1 and the jumps from 360 to 0 degrees are avoided. You can think of $v_{i}$ as a circle with a radius of 1.

### Trained model

The trained model is saved in `session_name.grid_cell_rnn_model.pt` in each session directory.

In [2]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import spikeA
from spikeA.Spike_train import Spike_train
from spikeA.Animal_pose import Animal_pose
from spikeA.Spatial_properties import Spatial_properties
from spikeA.Neuron import Simulated_place_cell, Simulated_grid_cell
from scipy.stats import multivariate_normal
from scipy.stats import poisson
from scipy.interpolate import interp1d
from tqdm import tqdm
from scipy import stats
from scipy import ndimage
from scipy.stats import wilcoxon, pearsonr
from scipy.signal import find_peaks
import torch
import numpy as np

from functions import *

# Break down of what goes into the model

<li>Input: Firing rate of many different grid cells as a function of time, split into sequences of 20 consequetive spike trains</li>
<li>Label: Location of the animal at that time point in toroidal coordinates</li>